# Max Flow Applications

The purpose of this assignment is to investigate applications of finding a Max Flow. The problem asks you to design and implement an algorithm for shipping a material between nodes with different supply and demand requirements.

* Please write code *only* in the bodies of the two functions, that is, following the TODO comments.
* Be careful not to use varibles defined outside of the functions.
* Breaking the two above rules may lead to 0 grades.

# Movie distribution

First solve Problem 1 from hw3-t. 

Now suppose a movie distributor would like to ship a copy of a film from CA to every other state. There are therefore 48 units to ship out of CA, and each other state receives 1 unit. 

The dataset contiguous-usa.dat lists the adjacent states in the US. Each line lists two adjacent states; thus AK and HI are omitted, but DC is included in the data. The following code reads in the graph of US states.

In [1]:
import networkx as nx
G = nx.Graph()

usa = open('contiguous-usa.dat')
for line in usa:
    s1, s2 = line.strip().split()
    G.add_edge(s1, s2)

We now encode the demands into the graph.

In [2]:
for state in G.nodes():
    if state != 'CA':
        G.nodes[state]['demand'] = 1
G.nodes['CA']['demand'] = -48

We will assign a uniform capacity of 16 to each edge. Since CA has only three adjacent states, this is the smallest possible uniform capacity that allows one to ship all 48 units out of CA. As we have created an undirected graph, and flows have directions, we first convert the graph to a directed graph.

In [3]:
G = nx.DiGraph(G)
uniform_capacity = 16
for (s1, s2) in G.edges():
    G.edges[s1, s2]['capacity'] = uniform_capacity

Complete the following function to implement your algorithm to find a flow with demands. Your function should work correctly for any input, not just the movie instance considered here. As always, you are encouraged to define auxiliary functions as needed for clarity.

In [4]:
def flow_with_demands(graph):
    """Computes a flow with demands over the given graph.
    
    Args:
        graph: A directed graph with nodes annotated with 'demand' properties and edges annotated with 'capacity' 
            properties.
        
    Returns:
        A dict of dicts containing the flow on each edge. For instance, flow[s1][s2] should provide the flow along
        edge (s1, s2).
        
    Raises:
        NetworkXUnfeasible: An error is thrown if there is no flow satisfying the demands.
    """    
    # TODO: Implement the function.
    
    R_graph = nx.Graph() # or whatever type of graph `G` is
    R_graph.add_edges_from(graph.edges())
    totalDemand=0
    totalSupply=0
    for state in list(graph.nodes()):
        #print(graph.nodes[state]['demand'])
        if graph.nodes[state]['demand'] >= 0:
            R_graph.add_edge(state, "SINK")
            R_graph.edges[state, "SINK"]['capacity'] = graph.nodes[state]['demand']
            totalDemand += graph.nodes[state]['demand']
        else:
            R_graph.add_edge("SOURCE",state)
            R_graph.edges["SOURCE",state]['capacity'] = -graph.nodes[state]['demand']
            totalSupply +=(-1* graph.nodes[state]['demand'])
    
    flow_value, flow_dict = nx.maximum_flow(R_graph, 'SOURCE', 'SINK')
    flow_dict = {i:flow_dict[i] for i in flow_dict if (i!="SOURCE" )}
    flow_dict = {i:flow_dict[i] for i in flow_dict if (i!="SINK" )}
    for j in flow_dict.keys():
        flow_dict[j]= {i:flow_dict[j][i] for i in flow_dict[j] if  i!="SINK"}
        flow_dict[j]= {i:flow_dict[j][i] for i in flow_dict[j] if  i!="SOURCE"}
    if totalSupply==totalDemand:
        return flow_dict
    else:
        raise nx.NetworkXUnfeasible('There is no flow satisfying the demands.')

To verify that your solution is correct, implement a function that computes the total flow into each node (which will be negative for supply nodes).

In [5]:
def divergence(flow):
    """Computes the total flow into each node according to the given flow dict.
    
    Args:
        flow: the flow dict recording flow between nodes.
        
    Returns:
        A dict of the net flow into each node.
    """
    # TODO: Implement the function.
    
    list_nodes=[]
    for node in flow:
        #if node not in ["SINK","SOURCE"]:
        list_nodes.append(node)
    dictResult = dict.fromkeys(list_nodes,0)
    for i  in flow:
        #if i not in ["SINK","SOURCE"]:
        for j in flow[i]:
            #if j not in ["SINK","SOURCE"]:
            dictResult[j]+=flow[i][j]
            dictResult[i]-=flow[i][j]

    return dictResult

The following code performs a sanity check on your function (but does not completely confirm correctness).

In [6]:
flow = flow_with_demands(G)
div = divergence(flow)
print("Flow satisfies all demands:", all(div[n] == G.node[n]['demand'] for n in G.nodes()))

Flow satisfies all demands: True
